# MLP Modeling - 3
지금 원핫 인코딩하는 작업을 Embedding layer로 넣어보는 시도

마구 생각:
 * vocab도 만들어야 하고.. tokenizing도 해야될 듯?
 * mlpregressor에서 keras로 바꾸는 것도 필요

In [1]:
import os
from datetime import datetime as dt

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
# import optuna

2023-04-25 16:36:07.284534: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 16:36:09.084566: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## load data

In [2]:
train_df = pd.read_csv("open/train.csv", index_col='ID')
test_df = pd.read_csv("open/test.csv", index_col='ID')
sample_submission_df = pd.read_csv("open/sample_submission.csv")

train_df.shape, test_df.shape, sample_submission_df.shape

((871393, 9), (159621, 8), (159621, 2))

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 871393 entries, TRAIN_000000 to TRAIN_871392
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   User-ID              871393 non-null  object 
 1   Book-ID              871393 non-null  object 
 2   Book-Rating          871393 non-null  int64  
 3   Age                  871393 non-null  float64
 4   Location             871393 non-null  object 
 5   Book-Title           871393 non-null  object 
 6   Book-Author          871393 non-null  object 
 7   Year-Of-Publication  871393 non-null  float64
 8   Publisher            871393 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 66.5+ MB


## preprocess data

In [4]:
train_users = train_df['User-ID'].unique()
train_books = train_df['Book-ID'].unique()

len(train_users), len(train_books)

(83256, 243441)

In [5]:
test_users = test_df['User-ID'].unique()
test_books = test_df['Book-ID'].unique()

len(test_users), len(test_books)

(21909, 62333)

In [6]:
len(set(train_users) | set(test_users)), len(set(train_books) | set(test_books))

(92101, 270056)

새로운 유저, 새로운 책이 분명 있음

In [7]:
train_df.head()

,User-ID,Book-ID,Book-Rating,Age,Location,Book-Title,Book-Author,Year-Of-Publication,Publisher
ID,,,,,,,,,
TRAIN_000000,USER_00000,BOOK_044368,8,23.0,"sackville, new brunswick, canada",Road Taken,Rona Jaffe,2001.0,Mira
TRAIN_000001,USER_00000,BOOK_081205,8,23.0,"sackville, new brunswick, canada",Macbeth (New Penguin Shakespeare),William Shakespeare,1981.0,Penguin Books
TRAIN_000002,USER_00000,BOOK_086781,0,23.0,"sackville, new brunswick, canada",Waverley (Penguin English Library),Walter Scott,1981.0,Penguin Books
TRAIN_000003,USER_00000,BOOK_098622,0,23.0,"sackville, new brunswick, canada",Mother Earth Father Sky,Sue Harrison,1991.0,Avon
TRAIN_000004,USER_00000,BOOK_180810,8,23.0,"sackville, new brunswick, canada",She Who Remembers,Linda Lay Shuler,1989.0,Signet Book


User feature
 * Age: int
 * Location: string

Book feature
 * Book-Title: string
 * Book-Author: string
 * Year-Of-Pulication: int
 * Publisher: string

특징의 분해 이전에 label 형태로도 해볼 수 있음

# Feature Selection

In [8]:
features = test_df.columns
features

Index(['User-ID', 'Book-ID', 'Age', 'Location', 'Book-Title', 'Book-Author',
       'Year-Of-Publication', 'Publisher'],
      dtype='object')

In [9]:
continuous_features = ['Age', 'Year-Of-Publication']
categorical_features = ['Location', 'Book-Title', 'Book-Author', 'Publisher']

In [10]:
train_df['Location'] = train_df['Location'].str.replace(' ', '')
train_df['Location_country'] = train_df['Location'].apply(lambda x: x.split(',')[-1])
train_df['Location_city'] = train_df['Location'].apply(lambda x: x.split(',')[-2])
train_df.drop(columns='Location', inplace=True)
train_df.head()

,User-ID,Book-ID,Book-Rating,Age,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location_country,Location_city
ID,,,,,,,,,,
TRAIN_000000,USER_00000,BOOK_044368,8,23.0,Road Taken,Rona Jaffe,2001.0,Mira,canada,newbrunswick
TRAIN_000001,USER_00000,BOOK_081205,8,23.0,Macbeth (New Penguin Shakespeare),William Shakespeare,1981.0,Penguin Books,canada,newbrunswick
TRAIN_000002,USER_00000,BOOK_086781,0,23.0,Waverley (Penguin English Library),Walter Scott,1981.0,Penguin Books,canada,newbrunswick
TRAIN_000003,USER_00000,BOOK_098622,0,23.0,Mother Earth Father Sky,Sue Harrison,1991.0,Avon,canada,newbrunswick
TRAIN_000004,USER_00000,BOOK_180810,8,23.0,She Who Remembers,Linda Lay Shuler,1989.0,Signet Book,canada,newbrunswick


In [11]:
test_df['Location'] = test_df['Location'].str.replace(' ', '')
test_df['Location_country'] = test_df['Location'].apply(lambda x: x.split(',')[-1])
test_df['Location_city'] = test_df['Location'].apply(lambda x: x.split(',')[-2])
test_df.drop(columns='Location', inplace=True)
test_df.head()

,User-ID,Book-ID,Age,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location_country,Location_city
ID,,,,,,,,,
TEST_000000,USER_00008,BOOK_047966,37.0,Birds of Prey: A Novel of Suspense,J.A. Jance,2002.0,Avon,usa,ohio
TEST_000001,USER_00008,BOOK_119494,37.0,Midnight Voices,JOHN SAUL,2003.0,Ballantine Books,usa,ohio
TEST_000002,USER_00008,BOOK_151775,37.0,Breaking Free : A Prescription for Personal an...,David M. Noer,1996.0,Jossey-Bass,usa,ohio
TEST_000003,USER_00008,BOOK_176255,37.0,Bitter Harvest,Ann Rule,1999.0,Pocket,usa,ohio
TEST_000004,USER_00008,BOOK_187307,37.0,Embraced by the Light,Betty J. Eadie,1994.0,Bantam Books,usa,ohio


# encoding

for location_country, publisher

In [12]:
from sklearn.preprocessing import OneHotEncoder

In [13]:
cont_encoder = OneHotEncoder(handle_unknown='ignore')
train_country_array = cont_encoder.fit_transform(train_df[['Location_country']]).toarray()
test_country_array = cont_encoder.transform(test_df[['Location_country']]).toarray()

In [14]:
# pub_encoder = OneHotEncoder(handle_unknown='ignore')
# train_publisher_array = pub_encoder.fit_transform(train_df[['Publisher']]).toarray()
# test_publisher_array = pub_encoder.transform(train_df[['Publisher']]).toarray()

# scaling

In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
scaler = StandardScaler()
train_cont_array = scaler.fit_transform(train_df[continuous_features])
test_cont_array = scaler.transform(test_df[continuous_features])

# merge prcessed data

In [17]:
train_cont_array.shape, train_country_array.shape#, train_publisher_array.shape

((871393, 2), (871393, 243))

In [18]:
train_x = {
    'input_continuous': train_cont_array,
    'input_country': train_country_array,
}

test_x = {
    'input_continuous': test_cont_array,
    'input_country': test_country_array,
}

In [19]:
# train_x = np.concatenate([train_cont_array, train_df[['Location_country']].values], axis=1) #train_publisher_array
# test_x = np.concatenate([test_cont_array, test_df[['Location_country']].values], axis=1) #test_publisher_array

# train_x.shape, test_x.shape

In [20]:
train_y = train_df['Book-Rating'].values
train_y.shape

(871393,)

In [21]:
# train_array = np.concatenate([train_cont_array, train_country_array, train_publisher_array], axis=1)
# test_array = np.concatenate([test_cont_array, test_country_array, test_publisher_array], axis=1)

# model

In [ ]:
alpha = 0.001

In [22]:
# define input tensor
input_continuous = tf.keras.Input(shape=train_cont_array.shape[1], dtype=tf.float32)
input_country = tf.keras.Input(shape=train_country_array.shape[1], dtype=tf.float32)

# embedding layer
embedding_layer = tf.keras.layers.Embedding(
train_country_array.shape[1], 100)(input_country)

# concatenate
x = tf.keras.layers.Concatenate(axis=-1)([input_continuous, input_country])

# hidden layer
x = tf.keras.layers.Dense(
    64, activation='relu', 
    kernel_regularizer=tf.keras.regularizers.l2(alpha))(x)

# hidden layer
x = tf.keras.layers.Dense(
    16, activation='relu',
    kernel_regularizer=tf.keras.regularizers.l2(alpha))(x)

# hidden layer
x = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.Model([input_continuous, input_country], x)

2023-04-25 16:36:26.991319: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-25 16:36:27.008494: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-25 16:36:27.010721: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [23]:
learning_rate = 0.0001

In [ ]:
# define optimizer RMSProp used instead of RProp
optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)

# model compile
i_autorec.compile(optimizer=optimizer, loss='mse', metrics='mse')

In [21]:
mname = 'mlpregressor'

In [22]:
from sklearn.neural_network import MLPRegressor

In [23]:
mlp = MLPRegressor(
    warm_start=True, learning_rate='adaptive', 
    activation='relu', batch_size=100, verbose=1, 
    random_state=42, max_iter=500)

In [24]:
mlp.fit(train_x, train_y)

Iteration 1, loss = 7.33940769
Iteration 2, loss = 7.29654412
Iteration 3, loss = 7.28895794
Iteration 4, loss = 7.28246655
Iteration 5, loss = 7.27683403
Iteration 6, loss = 7.27128892
Iteration 7, loss = 7.26637750
Iteration 8, loss = 7.26009798
Iteration 9, loss = 7.25516507
Iteration 10, loss = 7.25052316
Iteration 11, loss = 7.24648508
Iteration 12, loss = 7.24270885
Iteration 13, loss = 7.23947332
Iteration 14, loss = 7.23720072
Iteration 15, loss = 7.23487028
Iteration 16, loss = 7.23264992
Iteration 17, loss = 7.23095062
Iteration 18, loss = 7.22934724
Iteration 19, loss = 7.22765679
Iteration 20, loss = 7.22667590
Iteration 21, loss = 7.22458144
Iteration 22, loss = 7.22405742
Iteration 23, loss = 7.22247995
Iteration 24, loss = 7.22228038
Iteration 25, loss = 7.22070622
Iteration 26, loss = 7.21946391
Iteration 27, loss = 7.21909872
Iteration 28, loss = 7.21821334
Iteration 29, loss = 7.21679018
Iteration 30, loss = 7.21656546
Iteration 31, loss = 7.21549345
Iteration 32, los

In [ ]:
train_pred = mlp.predict(train_x)
train_pred

array([3.35973996, 2.80756705, 2.80756705, ..., 3.55383282, 2.40314881,
       2.9251127 ])

In [ ]:
train_pred.shape

(871393,)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
rmse = mean_squared_error(train_y, train_pred, squared=False)
rmse

3.7896076664581213

In [38]:
test_pred = mlp.predict(test_x)
test_pred

array([2.53896857, 2.56751312, 2.36770129, ..., 3.19584419, 3.31259461,
       3.40015742])

In [40]:
sample_submission_df = pd.read_csv("open/sample_submission.csv")
sample_submission_df

,ID,Book-Rating
0,TEST_000000,0
1,TEST_000001,0
2,TEST_000002,0
3,TEST_000003,0
4,TEST_000004,0
...,...,...
159616,TEST_159616,0
159617,TEST_159617,0
159618,TEST_159618,0
159619,TEST_159619,0


In [41]:
def make_report(template, test_pred, mname):
    template['Book-Rating'] = test_pred
    now = dt.strftime(dt.now(), '%y-%m-%d')
    template.to_csv(f'results/{mname}-{now}.csv', index=False)
    
make_report(sample_submission_df, test_pred, mname)

test_rmse = 3.8528787988